# data worth and related assessments

In this notebook, we will use outputs from previous notebooks (in particular `pestpp-glm_part1.ipynb`) to undertake data worth assessments based on first-order second-moment (FOSM) techniques. "Worth" is framed here in the context of the extent to which the uncertainty surrounding a model prediction of management interest is reduced through data collection.  Given that these anayses can help target and optimize data acquisition strategies, this is a concept that really resonates with decision makers.

In [ ]:
%matplotlib inline
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcParams['font.size']=12
import flopy
import pyemu


In [ ]:
m_d = "master_glm"

In [ ]:
pst = pyemu.Pst(os.path.join(m_d,"freyberg_pp.pst"))
print(pst.npar_adj)
pst.write_par_summary_table(filename="none")

first ingredient: parameter covariance matrix (representing prior uncertainty in this instance)

In [ ]:
cov = pyemu.Cov.from_binary(os.path.join(m_d,"prior_cov.jcb")).to_dataframe()
cov = cov.loc[pst.adj_par_names,pst.adj_par_names]
cov = pyemu.Cov.from_dataframe(cov)

second ingredient: jacobian matrix

In [ ]:
jco = os.path.join(m_d,"freyberg_pp.jcb")

the third ingredient--the (diagonal) noise covariance matrix--populated on-the-fly using weights when constructing the Schur object below...

In [ ]:
sc = pyemu.Schur(jco=jco,parcov=cov)

### there we have it--all computations done and contained within `sc`.  We will only be required to access different parts of `sc` below...

### Parameter uncertainty

First let's inspect the (approx) posterior parameter covariance matrix and the reduction in parameter uncertainty through "data assimilation", before mapping to forecasts... (note that this matrix is ${\it not}$ forecast-specific)

In [ ]:
sc.posterior_parameter.to_dataframe().sort_index(axis=1).iloc[100:105:,100:105]

We can see the posterior variance for each parameter along the diagonal. The off-diags are symmetric.

In [ ]:
par_sum = sc.get_parameter_summary().sort_values("percent_reduction",ascending=False)
par_sum.head()

In [ ]:
par_sum.loc[par_sum.index[:15],"percent_reduction"].plot(kind="bar",color="turquoise")

What have we achieved by "notionally calibrating" our model to 13 head and 1 stream flow observations? Which parameters are informed? Will they matter for the forecast of interest? Which ones are un-informed?

## Forecast uncertainty

In [ ]:
forecasts = sc.pst.pestpp_options['forecasts'].split(",")
forecasts

In [ ]:
df = sc.get_forecast_summary()
df

In [ ]:
# make a pretty plot 
fig = plt.figure()
ax = plt.subplot(111)
ax = df["percent_reduction"].plot(kind='bar',ax=ax,grid=True)
ax.set_ylabel("percent uncertainy\nreduction from calibration")
ax.set_xlabel("forecast")
plt.tight_layout()

Surprise, surprise... Some forecasts benefit from calibration, some do not! 

### Before moving onto data worth, let's look at the contribution of different parameters to forecast uncertainty

Parameter contributions to uncertainty are quantified by "fixing" parameters (or parameter groups) and observing the uncertainty reduction as a result. This approach is of course subject to some sizable assumptions--related to parameter representativeness. But it can be very informative. Let's do by group.

In [ ]:
par_contrib = sc.get_par_group_contribution()

In [ ]:
par_contrib.head()

In [ ]:
base = par_contrib.loc["base",:]
par_contrib = 100.0 * (base - par_contrib) / par_contrib

In [ ]:
for forecast in par_contrib.columns:
    fore_df = par_contrib.loc[:,forecast].copy()
    fore_df.sort_values(inplace=True, ascending=False)
    ax = fore_df.iloc[:10].plot(kind="bar",color="b")
    ax.set_title(forecast)
    ax.set_ylabel("percent variance reduction")
    plt.show()

### Data worth

### what is the worth of ${\it existing}$ observations?

What is happening under the hood is that we are recalculating the Schur complement without some of the observations to see how the posterior forecast uncertainty increases (wrt a "base" condition in which we have all observation data available).

In [ ]:
dw_rm = sc.get_removed_obs_importance()
dw_rm.head()

Here the ``base`` row contains the results of the Schur complement calculation (in terms of forecast uncertainty variance) using all observations.  

In [ ]:
# let's normalize to make more meaningful comparisons of data worth (unctainty variance reduction)
base = dw_rm.loc["base",:]
dw_rm = 100 * (dw_rm  - base) / base
dw_rm.head()

In [ ]:
for forecast in dw_rm.columns:
    fore_df = dw_rm.loc[:,forecast].copy()
    fore_df.sort_values(inplace=True, ascending=False)
    ax = fore_df.iloc[:10].plot(kind="bar",color="b")
    ax.set_title(forecast)
    ax.set_ylabel("percent variance increase")
    plt.show()

There is also an option to calculate the worth of observations by taking a "base" condition of zero observation (i.e., a priori) and calculating the reduction in uncertainty through adding observations to the dataset. 

In [ ]:
dw_ad = sc.get_added_obs_importance()
base = dw_ad.loc["base",:]
dw_ad = 100 * (base - dw_ad) / base
for forecast in dw_ad.columns:
    fore_df_ad = dw_ad.loc[:,forecast].copy()
    fore_df_ad.sort_values(inplace=True, ascending=False)
    ax = fore_df_ad.iloc[:10].plot(kind="bar",color="b")
    ax.set_title(forecast)
    ax.set_ylabel("percent variance decrease")
    plt.show()

Do these two approaches give the same answer? They shouldn't.. Why? Let's discuss..

### what is the worth of ${\it potential}$ observations? what data should we collect?

Recall we are "carrying" cell-by-cell heads, reach-based sfr flows, etc..

In [ ]:
z_obs = pst.observation_data.loc[(pst.observation_data.weight == 0),"obsnme"].tolist()
z_obs = [x for x in z_obs if x not in forecasts]  # less our forecasts
z_obs

We can therefore repeat above analysis for the observations that currently have zero weight by turning those observations "on".

#### Beware: calculating the Schur complement for all potential observation types and locations could take some time!! So we will sample to speed things up. You may need to further reduce the number of potential obs - you can do this by adding [0::2] to take every second element for example.

In [ ]:
new_obs = [x for x in z_obs if "hds_00" in x and x.endswith("_000")]  # all heads in top layer in first stress period
print("number of new potential head observation locations considered: {}".format(len(new_obs)))

In [ ]:
from datetime import datetime
start = datetime.now()
df_worth_new = sc.get_added_obs_importance(obslist_dict=new_obs, base_obslist=sc.pst.nnz_obs_names, reset_zero_weight=True)
print("took:",datetime.now() - start)

In [ ]:
df_worth_new.head()

### nice! now let's process a little bit and make some plots of (potential) data worth

In [ ]:
def worth_plot_prep(df):
    # some processing
    df_new_base = df.loc["base",:].copy()  # "base" row
    df_new_imax = df.apply(lambda x: df_new_base - x, axis=1).idxmax()  # obs with largest unc red for each pred
    df_new_worth = 100.0 * (df.apply(lambda x: df_new_base - x, axis=1) / df_new_base)  # normalizing like above
    
    # plot prep
    df_new_worth_plot = df_new_worth[df_new_worth.index != 'base'].copy()
    df_new_worth_plot.loc[:,'names'] = df_new_worth_plot.index
    names = df_new_worth_plot.names
    df_new_worth_plot.loc[:,"i"] = names.apply(lambda x: int(x[8:10]))
    df_new_worth_plot.loc[:,"j"] = names.apply(lambda x: int(x[11:14]))
    df_new_worth_plot.loc[:,'kper'] = names.apply(lambda x: int(x[-3:]))
    #df_new_worth_plot.head()
    
    return df_new_worth_plot, df_new_imax

In [ ]:
df_worth_new_plot, df_worth_new_imax = worth_plot_prep(df_worth_new)

In [ ]:
df_worth_new_plot.head()

In [ ]:
df_worth_new_imax  # which obs causes largest unc var reduction?

In [ ]:
df_worth_new_plot.drop(axis=1,labels=["part_status"],inplace=True) # drop "part_status"
df_worth_new_plot.head()

### plotting

In [ ]:
def plot_added_importance(df_worth_plot, ml, forecast_name=None, 
                          newlox=None,figsize=(12,7)):

    vmax = df_worth_plot[forecast_name].max()
    #vmax = df_worth_plot[forecast_name].max()
    
    fig = plt.figure(figsize=figsize)
    axlist = []
    # if new locations provided, plot them with their numbers
    if newlox:
        currx = []
        curry = []
        for i,clox in enumerate(newlox):
            crow = int(clox[8:10])
            ccol = int(clox[11:14])
            currx.append(ml.sr.xcentergrid[crow,ccol])
            curry.append(ml.sr.ycentergrid[crow,ccol])

    for SP in range(1):
        unc_array = np.zeros_like(ml.upw.hk[0].array) - 1
        df_worth_csp = df_worth_plot.groupby('kper').get_group(SP)
        for i,j,unc in zip(df_worth_csp.i,df_worth_csp.j,
                           df_worth_csp[forecast_name]):
            unc_array[i,j] = unc 
        unc_array[unc_array == -1] = np.NaN
        axlist.append(plt.subplot(111,aspect="equal"))
        cb = axlist[-1].imshow(unc_array,interpolation="nearest",
                               alpha=0.5,extent=ml.sr.get_extent(), 
                               vmin=0, vmax=vmax)
        if SP==0:
            plt.colorbar(cb,label="percent uncertainty reduction")
        
        # plot sfr
        #axlist[-1].scatter([14 for x in range(ml.nrow)],[x for x in range(ml.nrow)],marker='d',s=60)
        #axlist[-1].scatter([10],[10],marker='d',s=50,zorder=0)
        #ml.sfr.stress_period_data.plot(key="ireach",axes=[axlist[-1]])
       
        # plot the pumping wells
        #ml.wel.stress_period_data.plot(kper=1,mflay=2)

        if newlox:
            for i,(cx,cy,cobs) in enumerate(zip(currx, curry, newlox)):
                csp = int(cobs[-1])
                if csp == SP:
                    axlist[-1].plot(cx, cy, 'rd', mfc=None, ms=18, alpha=0.8)
                    axlist[-1].text(cx-50,cy-50,i, size='medium')
                
        # plot the location of the forecast if possible
        if forecast_name.startswith('hds'):
            i = int(forecast_name[8:10])
            j = int(forecast_name[11:14])
            forecast_x = ml.sr.xcentergrid[i,j]
            forecast_y = ml.sr.ycentergrid[i,j]
            axlist[-1].scatter(forecast_x, forecast_y, marker='o', s=600, 
                               alpha=0.5)

        plt.title('data worth for {0}'.format(forecast_name))
    return fig

In [ ]:
m = flopy.modflow.Modflow.load("freyberg.nam", model_ws=os.path.join(m_d))

In [ ]:
df_worth_new_plot['hds_00_013_002_001'].idxmax()

In [ ]:
for i in [x for x in forecasts if "part_status" not in x]:
    fig = plot_added_importance(df_worth_plot=df_worth_new_plot, ml=m, 
                                forecast_name=i)
    #fig.savefig('add_worth_{}.pdf'.format(i))

## "next most" important observation

This is what we would ultimately like to know... Takes into account what we already know through incrementally making additional observations. For example, consider making an observation in the middle of the zone of highest worth. Where should we subsequently collect data? 

Let's just use the same potential observation list for now (the head in every top-layer cell) and evaluate which ones to collect, if we only had the budget for 5, in the context of the particle travel time prediction.

In [ ]:
start = datetime.now()
next_most_df = sc.next_most_important_added_obs(forecast='part_time',niter=5,obslist_dict=dict(zip(new_obs,new_obs)),
                                                base_obslist=sc.pst.nnz_obs_names,reset_zero_weight=True)
print("took:",datetime.now() - start)

In [ ]:
next_most_df.head()

In [ ]:
fig = plot_added_importance(df_worth_new_plot, m, 'part_time', 
                            newlox=next_most_df.best_obs.tolist())

In [ ]:
# for fun after class
for i in [x for x in forecasts if "part_status" not in x]:
    next_most_df = sc.next_most_important_added_obs(forecast=i,niter=5,obslist_dict=dict(zip(new_obs,new_obs)),
                                                    base_obslist=sc.pst.nnz_obs_names,reset_zero_weight=True)
    fig = plot_added_importance(df_worth_new_plot, m, forcast_name=i, 
                                newlox=next_most_df.best_obs.tolist())
    fig.savefig('next_best_5_worth_{}.pdf'.format(i))

### Note: an important assumption underpinning the above is that the model is able to fit observations to a level that is commensurate with measurement noise... Are we comfortable with this assumption? We will discuss this more in `pestpp-glm_part2.ipynb`

In [ ]:
# recall...
pst.observation_data.loc[pst.nnz_obs_names,:]

## an "extra": parameter identifiability